In [3]:
path = '/home/esr21/data_restore_202208212126/'
path = '/Users/ckgathi/Downloads/data_restore_202208212126/'
fname = path + 'vaccinationhistory.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split(',')
lines.pop(0)
del headers[0]
se_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split(',')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(line)
        print('line: ', len(line))
        print('headers: ', len(headers))
        print(headers)
        raise ValidationError('Line is not the right size')
    data = dict(zip(headers, line))
    se_data.append(data)
    count += 1

In [4]:
import pytz
import os, sys, csv
import django
from dateutil import parser
from django.conf import settings
from django.utils.timezone import make_aware
from django.db.models.fields import DateTimeField, DateField, BooleanField, PositiveIntegerField, IntegerField
from esr21_subject.models import InformedConsent, EligibilityConfirmation, VaccinationHistory
records = []
count = 0
with open('/home/esr21/data_restore_202208212126/vaccinationhistory.csv', newline='') as file:
    reader = csv.DictReader(file)
    for row in reader:
        row = dict(row)
        model_fields = VaccinationHistory._meta.fields
        for field in model_fields:
            field_name = field.name
            if getattr(field, 'choices'):
                choices = getattr(field, 'choices')
                field_value = [choice[0] for choice in choices if choice[1] == row.get(field_name)]
                if field_value:
                    row.update({f'{field_name}': field_value[0]})
                else:
                    row.update({f'{field_name}': None})
            elif isinstance(field, DateTimeField):
                try:
                    field_value = parser.parse(row.get(field_name))
                    field_value = field_value.astimezone(pytz.timezone(settings.TIME_ZONE))
                except parser.ParserError:
                    row.update({f'{field_name}': None})
                else:
                    row.update({f'{field_name}': field_value})
            elif isinstance(field, DateField):
                try:
                    field_value = parser.parse(row.get(field_name) or '').date()
                except parser.ParserError:
                    row.update({f'{field_name}': None})
                else:
                    row.update({f'{field_name}': field_value})
            elif isinstance(field, BooleanField):
                field_value = True if row.get(field_name) == 'True' else False
                row.update({f'{field_name}': field_value})
            elif isinstance(field, (PositiveIntegerField, IntegerField, )):
                try:
                    field_value = int(row.get(field_name) or '')
                except ValueError:
                    row.update({f'{field_name}': None})
                else:
                    row.update({f'{field_name}': field_value})
            elif field_name == 'site':
                site_name = row.get('site')
                try:
                    site = Site.objects.get(domain=site_name)
                except Site.DoesNotExist:
                    raise ValidationError('Site Does not exist')
                else:
                    row.update({f'{field_name}': site})
        records.append(row)

for record in records:
    del record['']
    obj = VaccinationHistory(**record)
    obj.save_base(raw=True)

SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.

In [5]:
data

{'created': '2022-08-19 09:25:59.597982+00:00',
 'modified': '2022-08-19 09:25:59.598006+00:00',
 'user_created': 'lbafana',
 'user_modified': '',
 'hostname_created': 'esr21',
 'hostname_modified': 'esr21',
 'revision': '',
 'device_created': '99',
 'device_modified': '99',
 'id': '1033ba5e-6eb4-410c-a8ab-d487ce92f232',
 'site': 'esr21',
 'subject_identifier': '150-040994234-7',
 'subject_identifier_as_pk': '2fea89ac-6e4f-46d2-8833-5deffec7ff10',
 'subject_identifier_aka': '',
 'slug': '150-040994234-7',
 'report_datetime': '2022-08-19 09:24:00+00:00',
 'received_vaccine': 'Yes',
 'dose_quantity': '2',
 'dose1_product_name': 'Pfizer-BioNTech',
 'dose1_product_other': '',
 'dose1_date': '2021-12-21',
 'dose2_product_name': 'Pfizer-BioNTech',
 'dose2_product_other': '',
 'dose2_date': '2022-01-18',
 'source_of_info': 'Documented evidence (Vaccine card/Vaccine passport/Facility based record)'}